# 한글-Claude-v2 Model: Bedrock with LangChain using a Prompt that includes Context

---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

## 소개

이 노트북에서는 고객 지원 엔지니어로부터 받은 고객 서비스 품질에 만족하지 못한 고객에게 이메일 응답을 생성하는 방법을 보여줍니다. 불만 고객으로부터 받은 실제 이메일의 내용을 제공하여 모델에 추가 컨텍스트를 제공합니다.

프롬프트의 추가 컨텍스트로 인해 이 노트북의 Anthropic Claude-v2 언어 모델에서 생성된 텍스트는 이전에 제로샷 프롬프트를 통해 생성된 콘텐츠보다 품질과 관련성이 훨씬 뛰어납니다.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html)은 언어 모델로 구동되는 애플리케이션을 개발하기 위한 프레임워크입니다. 이 프레임워크의 주요 측면을 통해 다양한 구성 요소를 함께 연결하여 고급 사용 사례를 생성함으로써 대규모 언어 모델을 보강할 수 있습니다.

이 노트북에서는 LangChain에서 제공하는 Bedrock API를 사용합니다. 이 예에서 사용된 프롬프트는 텍스트 생성 요청에 컨텍스트를 추가하기 위한 사용자 지정 LangChain 프롬프트 템플릿을 생성합니다.

**참고:** *이 노트북은 AWS 환경 내부 또는 외부에서 실행할 수 있습니다.*

#### 문맥
이전 예제 `01_zero_shot_generation.ipynb`에서는 LangChain 프레임워크를 사용하여 Amazon Bedrock API와 통신하는 방법을 살펴보았습니다. 이 노트북에서는 유사한 사용 사례에 대해 LangChain 프레임워크를 활용하기 위해 'PromptTemplates'의 도움으로 좀 더 복잡함을 추가하려고 합니다. 'PrompTemplates'를 사용하면 나중에 정보를 채울 수 있는 일반 셸을 만들고 다양한 시나리오를 기반으로 모델 출력을 얻을 수 있습니다.

이 노트북의 일부로 LangChain 프레임워크 내에서 Amazon Bedrock 통합의 사용과 'PromptTemplate'의 도움으로 텍스트를 생성하는 데 사용할 수 있는 방법을 살펴보겠습니다.

#### 패턴
추가 예제를 제공하지 않고 출력을 생성하기 위해 내부 모델에 대한 작업, 지침 및 입력으로 구성된 입력을 Amazon Bedrock API의 LangChain 구현에 제공하기만 하면 됩니다. 여기의 목적은 강력한 LLM이 당면한 작업을 쉽게 이해하고 매력적인 출력을 생성하는 방법을 보여주는 것입니다.

![](./images/bedrock_langchain.jpg)

#### 사용 사례
Amazon Bedrock에서 모델의 생성 기능을 시연하기 위해 이메일 생성 사용 사례를 살펴보겠습니다.

#### 페르소나
귀하는 AnyCompany의 고객 서비스 관리자인 권율이며 일부 고객은 고객 서비스에 만족하지 않고 고객 지원 엔지니어가 제공하는 서비스에 대해 부정적인 피드백을 제공하고 있습니다. 이제는 열악한 서비스에 대해 겸허히 사과하는 고객에게 응답하고 신뢰를 회복하고 싶습니다. 인간 친화적이고 이전 이메일 서신에서 고객의 정서에 맞게 개인화된 대량의 이메일을 생성하려면 LLM의 도움이 필요합니다.

#### 구현
이 사용 사례를 이행하기 위해 고객의 이전 이메일을 기반으로 감사 메모가 포함된 이메일을 생성하는 방법을 보여줍니다. Amazon Bedrock LangChain 통합을 사용하는 Amazon Titan Text Large 모델을 사용합니다.

## 1. Bedrock Client 생성

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [2]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


In [3]:
! pip list | grep langchain
! pip list | grep opensearch

langchain                             0.1.3
langchain-community                   0.0.15
langchain-core                        0.1.15
opensearch-py                         2.4.2


## 2. LangChain 통합을 사용하여 Bedrock 클라이언트 호출

langchain.llms 로 부터 Bedrock 클래스의 인스턴스를 만드는 것으로 시작하겠습니다. 이것은 Amazon Bedrock에서 사용할 수 있는 모델의 'model_id'를 예상합니다.

선택적으로 이전에 생성된 boto3 `client`와 `temperature`, `topP`, `maxTokenCount` 또는 `stopSequences`와 같은 매개변수를 보유할 수 있는 일부 `model_kwargs`를 전달할 수 있습니다(매개변수에 대한 자세한 내용은 Amazon Bedrock에서 탐색할 수 있음)


다른 모델은 다른 `model_kwargs`를 지원합니다.

In [4]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {
    "max_tokens_to_sample": 4096, 
    "temperature":0.5,
    "top_k":250,
    "top_p":1,
    "stop_sequences": ["\n\nHuman"]
}

textgen_llm = Bedrock(
    model_id = bedrock_info.get_model_id(model_name="Claude-V2"),
    client = boto3_bedrock, 
    model_kwargs = inference_modifier 
)

## 3. LangChain 사용자 정의 프롬프트 템플릿 생성

프롬프트에 대한 템플릿을 생성하여 실행할 때마다 다른 입력 변수를 전달할 수 있습니다. 이는 데이터베이스에서 가져올 수 있는 다른 입력 변수를 사용하여 콘텐츠를 생성해야 할 때 유용합니다.

이전에는 프롬프트를 하드코딩했습니다. 유사한 부정적인 피드백을 보내는 여러 고객이 있고 이제 각 고객의 이메일을 사용하고 사과로 응답하지만 응답을 약간 개인화하려는 경우가 있을 수 있습니다. 다음 셀에서는 이 패턴을 달성하기 위해 'PromptTemplate'을 만드는 방법을 살펴봅니다.

In [5]:
from langchain import PromptTemplate

multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""서비스 관리자 {customerServiceManager}가 {customerName}에게 보내는 사과 이메일을 작성합니다.
   고객으로부터 받은 다음 피드백에 대한 응답: {feedbackFromCustomer}.
   """
)


# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="권율", 
                                 customerName="이순신", 
                                 feedbackFromCustomer="""안녕하세요 권율님,
     귀하의 고객 지원팀에 전화했을 때의 최근 경험에 매우 실망했습니다.
     나는 즉시 전화를 받을 것으로 예상했지만 전화를 받는 데 3일이 걸렸습니다.
     문제를 해결하기 위한 첫 번째 제안이 올바르지 않았습니다. 결국 문제는 3일 만에 해결 되었습니다.
     우리는 제공된 응답에 매우 만족하지 않으며 다른 곳에서 비즈니스를 수행하는 것을 고려할 수 있습니다.
     """
     )


In [6]:
print("prompt: \n", prompt)

prompt: 
 서비스 관리자 권율가 이순신에게 보내는 사과 이메일을 작성합니다.
   고객으로부터 받은 다음 피드백에 대한 응답: 안녕하세요 권율님,
     귀하의 고객 지원팀에 전화했을 때의 최근 경험에 매우 실망했습니다.
     나는 즉시 전화를 받을 것으로 예상했지만 전화를 받는 데 3일이 걸렸습니다.
     문제를 해결하기 위한 첫 번째 제안이 올바르지 않았습니다. 결국 문제는 3일 만에 해결 되었습니다.
     우리는 제공된 응답에 매우 만족하지 않으며 다른 곳에서 비즈니스를 수행하는 것을 고려할 수 있습니다.
     .
   


In [7]:
HUMAN = "\n\nHuman:"
ASSISTANT = "\n\nAssistant:"
prompt = HUMAN + prompt + ASSISTANT
# print(prompt)

In [8]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

Our prompt has 284 tokens


## 4.프로프트 실행
프롬프트 템플릿을 사용하여 호출하고 선별된 응답을 다시 볼 수 있습니다.

In [9]:
response = textgen_llm(prompt)

email = response[response.index('\n')+1:]

print_ww(colored(email, "green"))

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



이번 고객 응대 과정에서 부족했던 점에 대해 진심으로 사과드립니다. 고객님의 피드백을 주셔서 정말 감사드립니다.

우리 서비스팀은 고객님의 요청에 신속하고 정확하게 응대하는 것이 가장 중요하다는 것을 잘 알고 있습니다. 제가 고객님의 경험을 통해 배운 점은 우리가 더욱 빠르고 효과적으로 응대할
수 있도록 해야 한다는 것입니다.

앞으로 우리 서비스팀은 고객님의 요청에 24시간 이내에 응대하고, 문제 해결을 위한 최초 제안의 정확성을 높이기 위해 노력하겠습니다.

고객님의 지속적인 성원에 감사드리며, 우리 서비스의 질을 높이기 위해 최선을 다하겠습니다. 고객님의 소중한 의견이 우리 서비스 개선의 원동력이 될 것입니다.

다시 한번 진심으로 사과드리며, 앞으로 고객님의 기대에 부응하는 서비스를 제공할 수 있도록 하겠습니다.

감사합니다.


## 5. 개인화된 제품 설명서 생성

In [16]:
multi_var_prompt = PromptTemplate(
    input_variables=["customer_name", "keyword_01", "keyword_02", "keyword_03"], 
    template=""" 당신은 향수 전문가 입니다. 아래 고객 {customer_name} 님이 제공한 키워드를 가지고, 고객을 위한 향수 상품 리스트인 <perfume_list> 에서 향수를 하나 골라주고, 
                제품 설명서를 작성해주세요. 제품 설명서에는 고객 이름을 꼭 써주세요.
                고객으로부터 제공 받은 키워드: {keyword_01}, {keyword_02}, {keyword_03}

   <perfume_list>
   (A) 러브포이즌 포맨 페로몬 남자향수 오드 퍼퓸
   (B) 오드 퍼퓸 미니 바닐라 러스트 향수
   (C) 인써티 24시간지속 데일리 향수 100ml 17종 지속력좋은 남자 여자 니치 드레스퍼퓸 섬유향수
   (D) 뮤스끄블랑 [화이트머스크 은은하고 예쁜 머스크향 여성 명품 니치 타입 향수
   (E) 여름 향수인 르빠겐조 (L'eau par Kenzo) 는 젊은 여성을 위한 시원하고 달콤한 향수
   </perfume_list>
   """
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customer_name="문수아", 
                                 keyword_01="휴가", 
                                 keyword_02="바다", 
                                 keyword_03="친구"
)

prompt = HUMAN + prompt + ASSISTANT

In [17]:
print("prompt: \n", prompt)

prompt: 
 

Human: 당신은 향수 전문가 입니다. 아래 고객 문수아 님이 제공한 키워드를 가지고, 고객을 위한 향수 상품 리스트인 <perfume_list> 에서 향수를 하나 골라주고, 
                제품 설명서를 작성해주세요. 제품 설명서에는 고객 이름을 꼭 써주세요.
                고객으로부터 제공 받은 키워드: 휴가, 바다, 친구

   <perfume_list>
   (A) 러브포이즌 포맨 페로몬 남자향수 오드 퍼퓸
   (B) 오드 퍼퓸 미니 바닐라 러스트 향수
   (C) 인써티 24시간지속 데일리 향수 100ml 17종 지속력좋은 남자 여자 니치 드레스퍼퓸 섬유향수
   (D) 뮤스끄블랑 [화이트머스크 은은하고 예쁜 머스크향 여성 명품 니치 타입 향수
   (E) 여름 향수인 르빠겐조 (L'eau par Kenzo) 는 젊은 여성을 위한 시원하고 달콤한 향수
   </perfume_list>
   

Assistant:


In [18]:

response = textgen_llm(prompt)

email = response[response.index('\n')+1:]

print_ww(colored(email, "green"))


제가 문수아님의 키워드인 '휴가, 바다, 친구'를 고려하여 <perfume_list>에서 'L'eau par Kenzo' 를 추천드리겠습니다.

L'eau par Kenzo는 시트러스 계열의 상큼하고 청량한 향이 특징인 향수로, 바닷가 휴가를 가기에 적합한 제품이라 생각됩니다. 휴양지에서 친구들과 함께 사용하기에도 좋은
향기입니다.

문수아님의 여름 바캉스를 더욱 즐겁고 기억에 남는 시간으로 만들어줄 이 향수를 추천드립니다. 사용시 주의사항이나 다른 문의사항이 있으시면 언제든지 연락주세요. 좋은 하루
되세요!


## 6. 요약

결론적으로 우리는 컨텍스트 없이 LLM을 호출하면 원하는 결과를 얻지 못할 수 있음을 배웠습니다. 컨텍스트를 추가하고 프롬프트 템플릿을 추가로 사용하여 LLM의 출력을 제한함으로써 원하는 출력을 성공적으로 얻을 수 있습니다.